# Login HuggingFace to Upload Model

In [1]:
# !git config --global credential.helper store

# from huggingface_hub import notebook_login
# notebook_login()

# 1. Pre-processing

In [2]:
import transformers
print(transformers.__version__)

4.17.0


In [3]:
from datasets import load_dataset, load_metric, DatasetDict, Dataset
metric = load_metric("sacrebleu")

In [4]:
import pandas as pd
def pre_process_from_csv(path, n_row=100000):
    df_5M = pd.read_csv(path, nrows=n_row)
    list_5M = df_5M.to_dict('records')
    list_sub = ['LST_Corpus']*len(list_5M)
    dict_5M = pd.DataFrame({"translation": list_5M, "subdataset": list_sub})
    return dict_5M

In [10]:
raw_datasets = pre_process_from_csv('TEST_dataset_5M.csv', 1000000)

cut_datasets = DatasetDict()
cut_datasets = Dataset.from_pandas(raw_datasets, split="train+validation").train_test_split(0.2)

In [11]:
cut_datasets

DatasetDict({
    train: Dataset({
        features: ['translation', 'subdataset'],
        num_rows: 800000
    })
    test: Dataset({
        features: ['translation', 'subdataset'],
        num_rows: 200000
    })
})

In [15]:
# ?cut_datasets.push_to_hub

In [13]:
# cut_datasets.push_to_hub(repo_id="huak95/TNANA_2")

In [14]:
cut_datasets.save_to_disk('fuck_thth')

Flattening the indices:   0%|          | 0/800 [00:00<?, ?ba/s]

Flattening the indices:   0%|          | 0/200 [00:00<?, ?ba/s]

In [16]:
from transformers import AutoTokenizer
model_checkpoint = "Helsinki-NLP/opus-mt-th-en"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [17]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "translate English to Romanian: "
else:
    prefix = ""

In [18]:
max_input_len  = 128
max_target_len = 128

source_lang = 'th'
target_lang = 'en'

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples['translation']]
    targets = []
    for ex_ in examples['translation']:
        ex = ex_[source_lang]
        if ex is not None:
            targets.append(ex)
    
    model_inputs = tokenizer(inputs, max_length=max_input_len, truncation=True) # Pad to longest word (128 char)

    with tokenizer.as_target_tokenizer():
        try:
            labels = tokenizer(targets, max_length=max_target_len, truncation=True)
        except:
            print('targets: ', targets)

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

In [24]:
tokenized_datasets = cut_datasets.map(preprocess_function, batched=True)

  0%|          | 0/80 [00:00<?, ?ba/s]

  0%|          | 0/20 [00:00<?, ?ba/s]

In [25]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['translation', 'subdataset', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 80000
    })
    test: Dataset({
        features: ['translation', 'subdataset', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 20000
    })
})

# 2. Using Translation Model

### Using AutoModel (With model selection)

In [26]:
# import os
# os.listdir()

In [27]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
model_pretrain = "Helsinki-NLP/opus-mt-th-en"
# model_for_predict = "./opus-mt-th-en-finetuned-5k-th-to-en"
model_for_predict = "huak95/opus-mt-th-en-finetuned-5k-th-to-en"

model_pt = AutoModelForSeq2SeqLM.from_pretrained(model_pretrain)
model    = AutoModelForSeq2SeqLM.from_pretrained(model_for_predict)
tokenizer = AutoTokenizer.from_pretrained(model_pretrain)

Downloading:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/290M [00:00<?, ?B/s]

In [28]:
# !git lfs track "opus-mt-th-en-finetuned-5k-th-to-en/*"

# model.push_to_hub('huak95/opus-mt-th-en-finetuned-5k-th-to-en')
# # tokenizer.push_to_hub('huak95/opus-mt-th-en-finetuned-5k-th-to-en')

In [59]:
def harry_translate(input_texts, model=model_pt):
    inputs = tokenizer(input_texts['th'], return_tensors = "pt")

    outputs = model.generate(inputs["input_ids"],
                             max_length=40,
                             num_beams=4,
                             early_stopping=True)

    return tokenizer.decode(outputs[0]).replace('<pad> ',''), input_texts['en']

In [45]:
tokenized_datasets['test']['translation'][0:2]

[{'en': 'he wears glasses.', 'th': 'เขา ใส่ แว่น .'},
 {'en': 'kate this is kaoru', 'th': 'เค ท นี่ คือ คา โอ รุ'}]

In [47]:
for i in range(100,110):
    text_ = tokenized_datasets['test']['translation'][i]
    thai_text = text_['th']
    pred_text_pt, real_text = harry_translate(text_, model_pt)
    pred_text, real_text    = harry_translate(text_, model)
    
#     m_score = compute_metrics.compute(predictions=[pred_text], references=[real_text])

    print('thai_text:',  thai_text)
    print('real_text: ', real_text)
    print('pred_text: ', pred_text)
    print('pre_train: ', pred_text_pt)
    print()

thai_text: เอา ล่ะ
real_text:  here we go
pred_text:  เอา และ
pre_train:  All right.

thai_text: ฉัน ไป ทำ ธุระ ที่นั่น ทุกๆ สาม เดือน .
real_text:  i go there on business every three months.
pred_text:  ฉัน ไป ทํา ส่วน ที่นั่นที่ ทุกหลัง สาม 
pre_train:  I do business there every three months.

thai_text: โปรด ตัด ผม ออก จาก แผนการ ของ คุณ ที่ จะไป เล่น สกี ใน วัน สุดสัปดาห์ นี้
real_text:  please count me out of your plans to go skiing for the weekend .
pred_text:  โปรด ตั้ง ผม ออก จาก แรงการ ของ คุณ ที่
pre_train:  Please cut me off from your plans to go skiing this weekend

thai_text: ขอบคุณ กรุณา มา อีก
real_text:  thank you very much please come again
pred_text:  ขอบคุณ กรุณา มา อีก
pre_train:  Thank you. Please come again.

thai_text: เม็ด เหงื่อ ผุด ขึ้น บน หน้าผาก ของ เธอ
real_text:  beads of perspiration gathered in her on her forehead
pred_text:  และ น้ําหรับ แข่ ขึ้น บน หมาย ของ เธอ
pre_train:  The sweatdrops on her forehead

thai_text: วาด อิสระ
real_text:  free hand
pred_

# Test Metrics 

In [66]:
def harry_translate_from_text(input_texts, model=model_pt):
    inputs = tokenizer(input_texts, return_tensors = "pt")

    outputs = model.generate(inputs["input_ids"],
                             max_length=40,
                             num_beams=4,
                             early_stopping=True)

    return tokenizer.decode(outputs[0]).replace('<pad> ','')

In [149]:
import datasets
metric = datasets.load_metric('sacrebleu')

In [151]:
from tqdm.autonotebook import tqdm

In [152]:

evaluation_df = pd.DataFrame(cut_datasets['test']['translation'][0:1000]).dropna(subset='en')
evaluation_dataset = evaluation_df.to_numpy()

model_inputs, gold_refs, model_preds = [], [], []
for gold_references, model_input in tqdm(evaluation_dataset):
    
    model_pred = harry_translate_from_text(model_input, model)
#     print(model_input, "__", gold_ref, "__", model_pred)
    
    model_preds.append(model_pred)
    
#     metric.add_batch(predictions=model_predictions.split(' '), references=[gold_references.split(' ')])
# final_score = metric.compute()

  0%|          | 0/951 [00:00<?, ?it/s]

ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [155]:
import copy
test_bleu_df = copy.deepcopy(evaluation_df.iloc[0:len(model_preds)])
test_bleu_df['pred'] = model_preds
test_bleu_df

,en,th,pred
0,effort without effect,ความ พยายาม ที่ ไร้ ผล,a futile attempt
1,what colors did you have in mind,คุณ มี สี อะไร อยู่ ในใจ,what color do you have in mind
2,my apartment isn't that big.,อพาร์ตเมนต์ ของ ฉัน ไม่ ได้ ใหญ่ ขนาด นั้น .,my apartment isn't that big.
3,what can i spread on this bread,ฉัน สามารถ ทา อะไร บน ขนมปัง นี้ ได้,can i put anything on this bread
4,the wall stood fast,กำแพง ตั้งตระหง่าน,a towering wall
...,...,...,...
760,Some experts would say that those words seem m...,ผู้ ​ เชี่ยวชาญ ​ บาง ​ คน ​ คง ​ จะ ​ บอก ​ ว...,some experts would say that those words seem t...
761,aim high,ตั้ง เป้าหมาย ไว้ สูง,set a high goal
762,Rain Forests I read with great interest your s...,จาก ​ ผู้ ​ อ่าน ​ ของ ​ เรา,from our reader
763,could you put me through to mr murphy please,คุณ สามารถ นำ ฉัน ไปยัง คุณ เมอ ร ์ ฟ ี ่ ได้ไ...,could you take me to mr. murf please


In [82]:
!pip install sacrebleu

def cal_bleu(predict, ground_truth):
  """
  this function calculate bleu score between prediction and ground truth
  predict <List> : list of prediction string ["str1","str2","str3", ...]
  ground_truth <List> : list of groundtruth string ["gt1","gt2","gt3", ...]
  """
  from sacrebleu.metrics import BLEU
  bleu = BLEU()
  res = score = bleu.corpus_score(ground_truth, [predict])
  return res

refs = ['The dog bit the man.', 'It was not unexpected.', 'The man bit him first.',]
sys = ['The dog bit the man.', "It wasn't surprising.", 'The man had just bitten him.']

cal_bleu(sys, refs)


BLEU = 45.07 70.6/42.9/36.4/37.5 (BP = 1.000 ratio = 1.000 hyp_len = 17 ref_len = 17)

In [164]:
test_bleu_df_small = test_bleu_df.dropna(subset='en')
test_bleu_df_small

,en,th,pred
0,effort without effect,ความ พยายาม ที่ ไร้ ผล,a futile attempt
1,what colors did you have in mind,คุณ มี สี อะไร อยู่ ในใจ,what color do you have in mind
2,my apartment isn't that big.,อพาร์ตเมนต์ ของ ฉัน ไม่ ได้ ใหญ่ ขนาด นั้น .,my apartment isn't that big.
3,what can i spread on this bread,ฉัน สามารถ ทา อะไร บน ขนมปัง นี้ ได้,can i put anything on this bread
4,the wall stood fast,กำแพง ตั้งตระหง่าน,a towering wall
...,...,...,...
760,Some experts would say that those words seem m...,ผู้ ​ เชี่ยวชาญ ​ บาง ​ คน ​ คง ​ จะ ​ บอก ​ ว...,some experts would say that those words seem t...
761,aim high,ตั้ง เป้าหมาย ไว้ สูง,set a high goal
762,Rain Forests I read with great interest your s...,จาก ​ ผู้ ​ อ่าน ​ ของ ​ เรา,from our reader
763,could you put me through to mr murphy please,คุณ สามารถ นำ ฉัน ไปยัง คุณ เมอ ร ์ ฟ ี ่ ได้ไ...,could you take me to mr. murf please


In [169]:
cal_bleu(test_bleu_df_small['pred'].to_list(), test_bleu_df_small['en'].to_list())

That's 100 lines that end in a tokenized period ('.')
It looks like you forgot to detokenize your test data, which may hurt your score.
If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.


BLEU = 20.65 45.0/24.8/15.9/10.3 (BP = 1.000 ratio = 1.176 hyp_len = 8056 ref_len = 6849)

# Some fucking git push

In [19]:
# !ls 

In [20]:
# cut_datasets.save_to_disk('fuck_thth')

In [21]:
# %%bash
# git lfs install
# git clone https://huggingface.co/datasets/huak95/TNANA
# # if you want to clone without large files – just their pointers
# # prepend your git clone with the following env var:
# GIT_LFS_SKIP_SMUDGE=1

In [22]:
# %%bash
# cd TNANA
# git add .
# git commit -m 'Add THANA'
# git config --global user.name "huak95"
# git push
# ls

In [24]:
# !git push